# install dependencies

In [1]:
!pip install hazm
!pip install nltk
!pip install tqdm

# import libraries

In [2]:
import pandas as pd
import json
from hazm import *
import tqdm
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pickle
from sklearn.preprocessing import normalize
import networkx as nx

# make dataframe from data set

In [4]:
news_dict = {}
with open('../data/news.json', "r", encoding="utf-8") as text_file:
        news_dict = json.loads(text_file.read())

df = pd.DataFrame.from_dict(news_dict, orient='index')
df

,title,subject,content
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...


# pre processing

In [5]:
normalizer = Normalizer()

normalized_news_dict = {}
for idx,value in tqdm.tqdm(news_dict.items()):
    normalized_news_dict[idx] = {"title" : normalizer.normalize(value['title']) ,"subject" : value["subject"], "content":value['content']}
  
news_df = pd.DataFrame.from_dict(normalized_news_dict, orient='index')

100%|██████████████████████████████████| 14876/14876 [00:00<00:00, 19846.79it/s]


In [8]:
new_stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('../custom_requirements/new_stopwords.txt','r','utf-8').readlines()]
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('../custom_requirements/stopwords.txt','r','utf-8').readlines()]

In [9]:
tokenized_words = [word_tokenize(_) for _ in news_df.title]
removed_tokenized_words = []
for per_content_words in tokenized_words:
  words = []
  for word in per_content_words:
    if word not in stopwords + new_stopwords:
      words.append(word)
  removed_tokenized_words.append(words)

news_df['word_tokenize'] = removed_tokenized_words
news_df

,title,subject,content,word_tokenize
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[هاشمی, هماهنگ, آزمون, خبرگان, حسن, روحانی, سا..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور مس...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...,"[اصلاح, قانون, اسلحه, کجا, کلید, خورد, ا, مامو..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...,"[ایران, قطعنامه‌های, آژانس, سکوت, نمی‌کند, غرب..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[اینفوگرافیک, اصلاحات, قانون, به‌کارگیری, اسلح..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[پیام, تبریک, رئیسی, رییس‌جمهور, فیلیپین]"
...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[عکس, ۲, ستاره, تیم, ملی, برترین, گلزنان, مقدم..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[عکس|, ۲۷, تیم, جام, جهانی, ۵, تیم, باقی, ماند..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[عکس, برد, پرگل, برزیل, شب, خداحافظی, کلمبیا, ..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[خداحافظی, غم, انگیز, کی, تیم, ملی, مصر, رنج, ..."


In [10]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt == 1:
        return stemmer.stem(tok)
    if opt == 2:
        return lemmatizer.lemmatize(tok)

In [11]:
opt = 2
lemmatize_stemming_words = []
for per_content_words in news_df.word_tokenize:
  words = []
  for word in per_content_words:
    words.append(get_lemma_set(word,opt))
  lemmatize_stemming_words.append(words)
news_df['word_lemmatize'] = lemmatize_stemming_words
news_df

,title,subject,content,word_tokenize,word_lemmatize
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[هاشمی, هماهنگ, آزمون, خبرگان, حسن, روحانی, سا...","[هاشم, هماهنگ, آزمون, خبرگان, حسن, روحانی, ساد..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور مس...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...,"[اصلاح, قانون, اسلحه, کجا, کلید, خورد, ا, مامو...","[اصلاح, قانون, اسلحه, کجا, کلید, خورد#خور, ا, ..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...,"[ایران, قطعنامه‌های, آژانس, سکوت, نمی‌کند, غرب...","[ایران, قطعنامه, آژانس, سکوت, کرد#کن, غرب, صدو..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[اینفوگرافیک, اصلاحات, قانون, به‌کارگیری, اسلح...","[اینفوگرافیک, اصلاحات, قانون, به‌کارگیری, اسلح..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[پیام, تبریک, رئیسی, رییس‌جمهور, فیلیپین]","[پیام, تبریک, رئیس, رییس‌جمهور, فیلیپین]"
...,...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[عکس, ۲, ستاره, تیم, ملی, برترین, گلزنان, مقدم...","[عکس, ۲, ستاره, تیم, ملی, بر, گلزنان, مقدمات, ..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[عکس|, ۲۷, تیم, جام, جهانی, ۵, تیم, باقی, ماند...","[عکس|, ۲۷, تیم, جام, جهانی, ۵, تیم, باقی, ماند..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[عکس, برد, پرگل, برزیل, شب, خداحافظی, کلمبیا, ...","[عکس, برد, پرگل, برزیل, شب, خداحافظی, کلمبیا, ..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[خداحافظی, غم, انگیز, کی, تیم, ملی, مصر, رنج, ...","[خداحافظی, غم, انگیز, کی, تیم, ملی, مصر, رنج, ..."


In [12]:
words = []
for row_words in news_df['word_lemmatize']:
  words.append(' '.join(row_words))
news_df['clean_text'] = words
news_df = news_df.drop(['word_lemmatize'],axis=1)
news_df

,title,subject,content,word_tokenize,clean_text
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[هاشمی, هماهنگ, آزمون, خبرگان, حسن, روحانی, سا...",هاشم هماهنگ آزمون خبرگان حسن روحانی ساده گرفت#گیر
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور مس...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...,"[اصلاح, قانون, اسلحه, کجا, کلید, خورد, ا, مامو...",اصلاح قانون اسلحه کجا کلید خورد#خور ا مامور مس...
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...,"[ایران, قطعنامه‌های, آژانس, سکوت, نمی‌کند, غرب...",ایران قطعنامه آژانس سکوت کرد#کن غرب صدور قطعنا...
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[اینفوگرافیک, اصلاحات, قانون, به‌کارگیری, اسلح...",اینفوگرافیک اصلاحات قانون به‌کارگیری اسلحه چیست
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[پیام, تبریک, رئیسی, رییس‌جمهور, فیلیپین]",پیام تبریک رئیس رییس‌جمهور فیلیپین
...,...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[عکس, ۲, ستاره, تیم, ملی, برترین, گلزنان, مقدم...",عکس ۲ ستاره تیم ملی بر گلزنان مقدمات جام جهانی...
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[عکس|, ۲۷, تیم, جام, جهانی, ۵, تیم, باقی, ماند...",عکس| ۲۷ تیم جام جهانی ۵ تیم باقی مانده قاره بو...
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[عکس, برد, پرگل, برزیل, شب, خداحافظی, کلمبیا, ...",عکس برد پرگل برزیل شب خداحافظی کلمبیا شیلی جام...
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[خداحافظی, غم, انگیز, کی, تیم, ملی, مصر, رنج, ...",خداحافظی غم انگیز کی تیم ملی مصر رنج مردم کشور...


## take an input

In [13]:
given_subject = input()

ورزش


## split dataframe with given input

In [14]:
selected_df = news_df[news_df["subject"] == given_subject] 

In [15]:
pre_processed_news_title = selected_df['clean_text'].tolist()
selcted_removed_tokenized_words = selected_df['word_tokenize'].tolist()

## create similarity matrix

In [16]:
news_num = len(selected_df)
words_set = [set(ls) for ls in selcted_removed_tokenized_words]
similarity_mat = np.zeros((news_num, news_num), dtype=float)
treshold = 4

for i in range(news_num):
    for j in range(news_num):
        intersect_len = len(words_set[i].intersection(words_set[j]))
        if intersect_len < treshold or i == j:
            similarity_mat[i][j] = 0
        else:
            similarity_mat[i][j] = (intersect_len ** (1.2)) / len(words_set[i])

## normalizing similarity matrix

In [17]:
similarity_mat_normalized = normalize(similarity_mat, norm='l1')

## create graph

In [18]:
graph = nx.from_numpy_array(similarity_mat_normalized)

In [19]:
nx.write_gml(graph, "../models/link_analysis/graph.gml")

In [20]:
graph_loaded = nx.read_gml('../models/link_analysis/graph.gml')

## check similarity matrix for symmetric

In [41]:
np.allclose(similarity_mat_normalized, similarity_mat_normalized.T, rtol=1e-05)

False

# page rank algorithm

## implement page rank algorithm

In [21]:
page_rank = nx.pagerank(graph_loaded, alpha=0.9)

## function for getting most important news

In [22]:
def get_top_n_news(values, n=5):
    top_n = np.argsort(list(values))[::-1][:n]
    return top_n

### page rank results

In [23]:
for news_id in get_top_n_news(page_rank.values()):
    print(selected_df.iloc[news_id]['title'])
    print('-------------------------------')

ادعای خبرساز بازیکن جنجالی عراق؛ به تیم ملی فوتبال ایران دعوت شدم! | همسرم ایرانی است و از لیگ این کشور باز هم پیشنهاد دارم
-------------------------------
ببینید | شادی مهدی طارمی با پرچم ایران در جشن قهرمانی پورتو | ژست خاص ستاره ایرانی مقابل هواداران
-------------------------------
ببینید | شوخی ترسناک لورکوزن با سردار آزمون | واکنش ستاره ایرانی به اقدام باشگاه آلمانی
-------------------------------
رای دیدار جنجالی لیگ برتر اعلام شد | بازی پرسپولیس - تراکتور ۳ بر صفر و دو بازیکن محروم شدند
-------------------------------
زمان عجیب اعلام رای بازی جنجالی لیگ برتر | ۲ بازیکن پرسپولیس به کمیته انضباطی دعوت شدند
-------------------------------


# HITS algorithm

## implement HITS algorithm

In [24]:
hubs, authorities = nx.hits(graph_loaded)

/Users/nonaghazizadeh/opt/anaconda3/lib/python3.9/site-packages/networkx/algorithms/link_analysis/hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


### authorities results

In [25]:
for news_id in get_top_n_news(authorities.values()):
    print(selected_df.iloc[news_id]['title'])
    print('-------------------------------')

ادعای خبرساز بازیکن جنجالی عراق؛ به تیم ملی فوتبال ایران دعوت شدم! | همسرم ایرانی است و از لیگ این کشور باز هم پیشنهاد دارم
-------------------------------
تصمیم جنجالی ملی‌پوش ایرانی برای رفتن به آمریکا | واکنش مدیر تیم‌های ملی به اقدام خبرساز
-------------------------------
سوءقصد خبرساز در فوتبال ایران | واژگون شدن خودروی چهره جنجالی تصادفی نبود!
-------------------------------
پشت پرده خروج دختر وزنه‌بردار ایران از هتل تیم ملی و استوری خبرساز مادرش | ورزشکار جوان فقط مدالش را برد!
-------------------------------
بانوی ایرانی سرمربی تیم ملی عراق شد
-------------------------------


### hubs results

In [26]:
for news_id in get_top_n_news(hubs.values()):
    print(selected_df.iloc[news_id]['title'])
    print('-------------------------------')

ادعای خبرساز بازیکن جنجالی عراق؛ به تیم ملی فوتبال ایران دعوت شدم! | همسرم ایرانی است و از لیگ این کشور باز هم پیشنهاد دارم
-------------------------------
تصمیم جنجالی ملی‌پوش ایرانی برای رفتن به آمریکا | واکنش مدیر تیم‌های ملی به اقدام خبرساز
-------------------------------
سوءقصد خبرساز در فوتبال ایران | واژگون شدن خودروی چهره جنجالی تصادفی نبود!
-------------------------------
پشت پرده خروج دختر وزنه‌بردار ایران از هتل تیم ملی و استوری خبرساز مادرش | ورزشکار جوان فقط مدالش را برد!
-------------------------------
بانوی ایرانی سرمربی تیم ملی عراق شد
-------------------------------


# TF-IDF vectorizer similarity matrix

## creating vocabulary and tfidf vectorizer

In [48]:
vocabulary = set()
for doc in selected_df.clean_text:
    vocabulary.update(doc.split(' '))
vocabulary = list(vocabulary)

vectorizer = TfidfVectorizer(ngram_range=(1,2), vocabulary=vocabulary, stop_words=None, norm='l2')

## save vecotrizer and load 

In [49]:
pickle.dump(vectorizer, open("./vectorizer.pickle", "wb"))
vectorizer = pickle.load(open("./vectorizer.pickle", 'rb'))

In [50]:
doc_term = vectorizer.fit_transform([' '.join(x) for x in selcted_removed_tokenized_words])

/Users/nonaghazizadeh/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


## create tf-idf similarity matrix

In [51]:
tf_idf_similarity_mat = doc_term.dot(doc_term.T)

## normalizing similarity matrix

In [52]:
tf_idf_similarity_mat_normalized = normalize(tf_idf_similarity_mat, norm='l1')

## create tf-idf graph

In [53]:
tf_idf_graph = nx.from_numpy_matrix(tf_idf_similarity_mat_normalized.toarray())

# page rank algorihm on tf-idf

## implement page rank algorithm on tf-idf graph

In [54]:
tf_idf_page_rank = nx.pagerank(tf_idf_graph, alpha=0.9)

### page rank results on tf-idf graph

In [55]:
for news_id in get_top_n_news(tf_idf_page_rank.values()):
    print(selected_df.iloc[news_id]['title'])
    print('-------------------------------')

مجیدی گرانترین مربی تاریخ فوتبال ایران شد | رقم نجومی قرارداد سرمربی سابق استقلال
-------------------------------
۲ بازیکن استقلال و پرسپولیس به تیم ملی دعوت شدند
-------------------------------
عکس | اولین قهرمانی بچه‌های فوتبال ایران در آسیا | تمجید AFC از تیم کاپیتان سابق پرسپولیس
-------------------------------
هافبک پرسپولیس هم رفتنی شد | خداحافظی با سرخ‌ها بعد از بازگشت به ایران
-------------------------------
عکس | استوری خبرساز ستاره سابق پرسپولیس با طعنه به گل‌محمدی | خوش‌آمدگویی به بازیکن جدید سرخ‌ها
-------------------------------


# HITS algorihm on tf-idf

In [56]:
tf_idf_hubs, tf_idf_authorities = nx.hits(tf_idf_graph)

### authorities results on tf-idf graph

In [57]:
for news_id in get_top_n_news(tf_idf_authorities.values()):
    print(selected_df.iloc[news_id]['title'])
    print('-------------------------------')

۲ بازیکن استقلال و پرسپولیس به تیم ملی دعوت شدند
-------------------------------
مجیدی گرانترین مربی تاریخ فوتبال ایران شد | رقم نجومی قرارداد سرمربی سابق استقلال
-------------------------------
بازیکن سابق تیم ملی و استقلال درگذشت
-------------------------------
هافبک پرسپولیس هم رفتنی شد | خداحافظی با سرخ‌ها بعد از بازگشت به ایران
-------------------------------
عکس | استوری خبرساز ستاره سابق پرسپولیس با طعنه به گل‌محمدی | خوش‌آمدگویی به بازیکن جدید سرخ‌ها
-------------------------------


### hubs results on tf-idf graph

In [58]:
for news_id in get_top_n_news(tf_idf_hubs.values()):
    print(selected_df.iloc[news_id]['title'])
    print('-------------------------------')

۲ بازیکن استقلال و پرسپولیس به تیم ملی دعوت شدند
-------------------------------
مجیدی گرانترین مربی تاریخ فوتبال ایران شد | رقم نجومی قرارداد سرمربی سابق استقلال
-------------------------------
بازیکن سابق تیم ملی و استقلال درگذشت
-------------------------------
هافبک پرسپولیس هم رفتنی شد | خداحافظی با سرخ‌ها بعد از بازگشت به ایران
-------------------------------
عکس | استوری خبرساز ستاره سابق پرسپولیس با طعنه به گل‌محمدی | خوش‌آمدگویی به بازیکن جدید سرخ‌ها
-------------------------------
